# Set up

In [54]:
# import spotipy
# from spotipy import util
# from spotipy.oauth2 import SpotifyClientCredentials
from get_spotify_client import get_spotify_obj
from utility_functions import create_connection
from utility_functions  import execute_sql_query
import data_pull_functions as pull

`client_id`: client ID from your project app on the spotify's app [dashboard](https://developer.spotify.com/dashboard)

`client_secret`: client secret from your project app on the spotify's app [dashboard](https://developer.spotify.com/dashboard)

`client_credentials_manager`: This create's a client to access the spotify API.  Since no access token was created, only public data may be accessed and no user interaction is available.

`spotipy.Spotify`: This is an object to send requests to the spotify API

## Getting Spotify Client

In [55]:
sp = get_spotify_obj()

# Get the [top 50 global playist](https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7)

`playlist_tracks` - Grabs a spotify playlist as `dict`.  In this case, we grab the _global top 50_.  Please call the `items` kvp to get a list of the tracks contained in the playlist

In [56]:
# pulls a playlist object & returns the attribute 'items'. 'items' is the list of track objects for tracks in playlist
top_fifty_global_tracks = sp.playlist_tracks("https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7",
                                      fields='items')


top_fifty_global_tracks = top_fifty_global_tracks.get('items')



Current fields we need for playlist relation:

* `track_name` - The name of the track lol
* `track_id` - The track's id
* `album_name` - The name of the album the track is a part of
* `album_id` - The album's id
* `date_added` - `DateTime` this was loaded this tuple was loaded to our database
 

The following is an example of a track in our playlist.  Each track returns as a dict and we want to highlight the various information we oculd get.

In [18]:
track_one = top_fifty_global_tracks[1]
track_one.get('track').get('artists')
track_one

{'added_at': '2024-03-23T10:06:21Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
  'href': 'https://api.spotify.com/v1/users/',
  'id': '',
  'type': 'user',
  'uri': 'spotify:user:'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'},
     'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR',
     'id': '66CXWjxzNUsdJxJ2JdwvnR',
     'name': 'Ariana Grande',
     'type': 'artist',
     'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',

The track's `track` kvp is where we get the information of the `track_id`, `track`, `album` columns

In [6]:
print("Track Name: " + track_one.get('track').get('name'))
print("Track Id: " + track_one.get('track').get('id'))

Track Name: LA FALDA
Track Id: 7iUtQNMRB8ZkKC4AmEuCJC


the track's `album` kvp is where we can get the track's `album` and `album_id`

In [7]:
print("Album Name: " + track_one.get('track').get('album').get('name'))
print("Album Id: " + track_one.get('track').get('album').get('id'))

Album Name: LVEU: VIVE LA TUYA...NO LA MIA
Album Id: 1ZVJTCW5rxpHKNgsNL91J2


# Grabbing top 50 tracks in playlist

`get_global_top_fifty_data` returns a `list` of `tuple` where each tuple is a track data in the format `(track, track_id, album, album_id)`


In [8]:
def get_global_top_fifty_song_album_data(sp):
    playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
    top_fifty_global = sp.playlist_tracks(playlist_id)

# This is a list of all tracks in the playlist.  Track objects are dictionaries
    top_fifty_global_tracks = top_fifty_global.get("items")

    tuples = []
    
    for song in top_fifty_global_tracks:
        added_it = song.get('added_at')
        
        song_info = song.get('track')
        track = song_info.get('name')
        track_id = song_info.get('id')
    
        album_info = song_info.get('album')
        album = album_info.get('name')
        album_id = album_info.get('id')

        date_loaded = str(date.today())
    
        row = (track, track_id, album, album_id, date_loaded)
        tuples.append(row)

    return tuples

# Getting track data for track table

`get_all_song_ids` returns a list of all the unique tracks and their ids for the track table as a `dict`.

In [9]:
def get_all_song_ids(list_of_tracks):
    track_dict = {}
    ids = set()
    
    for track in list_of_tracks:
        track_id = track[1]
        track_name = track[0]
        
        if track_id not in ids:
            track_dict.update({track_id:track_name})
        else:
            continue
    return track_dict

In [10]:
#data = get_global_top_fifty_song_album_data(sp)

# Grabbing the artist data

`get_all_top_fifty_artists` returns the artist that related to all of the songs in a playlist

In [11]:
def get_all_top_fifty_artists(sp):
    playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
    top_fifty_global = sp.playlist_tracks(playlist_id)
    top_fifty_global_tracks = top_fifty_global.get("items")
    tuples = []

    for track in top_fifty_global_tracks:
        # This is a list of dicts containing where each dict is an artist's information
        track_artists = track.get('track').get('artists')
        track_artists_data = [(artist.get('id'), artist.get('name')) for artist in track_artists]
        tuples += track_artists_data

    unique_artists = set(tuples)
    tuples = list(tuples)
    return tuples

# Grabbing the album data

`get_all_top_fifty_albums` returns the artist that related to all of the songs in a playlist

In [12]:
def get_all_top_fifty_albums(sp):
    playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
    top_fifty_global = sp.playlist_tracks(playlist_id)
    top_fifty_global_tracks = top_fifty_global.get("items")
    tuples = []

    for track in top_fifty_global_tracks:
        # This is a list of dicts containing where each dict is an artist's information
        album_id = track.get('track').get('album').get('id')
        album_name = track.get('track').get('album').get('name')
        row =(album_id, album_name)
        tuples.append(row)
        
    return tuples

# Inserting data

## Insert Tracks

In [13]:
def insert_tracks(data, conn):
    get_current_data_query = """
    SELECT * FROM Tracks;
    """
    
    insert_query = """
    INSERT INTO Tracks (track_id, track_name) VALUES (?, ?)
    """
    
    cur = conn.cursor()
    with conn:
        
    # grab current data
        current_data = execute_sql_query(get_current_data_query, conn)
        current_data_set = set(current_data)
        data_set = set(data)
        new_data = data_set - current_data_set
        new_rows = list(new_data)
    # to insert data
        cur.executemany(insert_query, new_rows)
        conn.commit()
        

In [14]:
def insert_albums(data, conn):
    get_current_data_query = """
    SELECT * FROM Albums;
    """
    
    insert_query = """
    INSERT INTO Albums (album_id, album_name) VALUES (?, ?)
    """
    
    cur = conn.cursor()
    with conn:
        
    # grab current data
        current_data = execute_sql_query(get_current_data_query, conn)
        current_data_set = set(current_data)
        data_set = set(data)
        new_data = data_set - current_data_set
        new_rows = list(new_data)
    # to insert data
        cur.executemany(insert_query, new_rows)
        conn.commit()

In [15]:
def insert_artists(data, conn):
    get_current_data_query = """
    SELECT * FROM Artists;
    """
    
    insert_query = """
    INSERT INTO Artists (artist_id, artist_name) VALUES (?, ?)
    """
    
    cur = conn.cursor()
    with conn:
        
    # grab current data
        current_data = execute_sql_query(get_current_data_query, conn)
        current_data_set = set(current_data)
        data_set = set(data)
        new_data = data_set - current_data_set
        new_rows = list(new_data)
    # to insert data
        cur.executemany(insert_query, new_rows)
        conn.commit()

In [16]:
def insert_playlist(data, conn):
    get_current_data_query = """
    SELECT * FROM Playlist;
    """
    
    insert_query = """
    INSERT INTO Playlist (track_id, track_name, album_id, album_name, date_loaded) VALUES (?, ?, ?, ?, ?)
    """
    
    cur = conn.cursor()
    with conn:
        
    # grab current data
        current_data = execute_sql_query(get_current_data_query, conn)
        current_data_set = set(current_data)
        data_set = set(data)
        new_data = data_set - current_data_set
        new_rows = list(new_data)
    # to insert data
        cur.executemany(insert_query, new_rows)
        conn.commit()

In [17]:
conn = create_connection('SpotifyData.db')
playlist = pull.get_global_top_fifty_playlist(sp)
insert_playlist(data=playlist, conn=conn)


# New Code
Each function below will grab data for a particular table or are helper functions to grab data for a particular table

## Raw Data Pull

`get_playlist_data` gets the data raw data for a playlist.   The raw data includes the name of the playlists and all information related to tracks, artists, and albums with the exception of audio analysis, market code definitions, key signature encodings, and other audio analysis metrics.

In [5]:
def get_playlist_data(sp, link):
    """
    inputs:
        sp: A spotify object
        link: url to the spotify playlist
    outputs:
        dictionary of two keys (name, followers, items)
        'name' is the name of the playlist
        'items' is a list of attributes to upload in playlists 
        Track objects are one of those attributes
        Track objects are just dict
    """

    playlist = sp.playlist(link)
    playlist_name = playlist.get('name')
    playlist_items = playlist.get('tracks').get('items')
    playlist_data = {'name': playlist_name, 'items': playlist_items}
    return playlist_data

`get_audio_features` gets the audio features of a track or list of tracks

In [6]:
def get_audio_features(sp, tracks: list[str]):
    """
    inputs:
        sp: A spotify object
        tracks: A list of spotify track URIs to return audio features for (maximum number of tracks to pull data in one call is 100)
    outputs:
        a dictionary with track IDs as keys and audio features as values
    """
    if len(tracks) > 100:
        raise Exception("Too many tracks in 'tracks' argument")
    else:
        data = sp.audio_features(tracks)
        updated_data = {}
        for feature in data:
            track_id = feature.pop("id")
            updated_data.update({track_id:feature})
        return updated_data
    

## Get Table Data

In [21]:
test_data = get_playlist_data(sp, 'https://open.spotify.com/playlist/37i9dQZEVXbLRQDuF5jeBp?si=778db61081ae460f')

`get_playlist_table_data` is a function to get the data for `Playlist`

In [8]:
def get_playlist_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Playlist
    """
# Name of playlist
    name = playlist_data.get("name")
# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        
        track_id = track.get('id')
        track_name = track.get('name')
        album_id = track.get('album').get('id')
        album_name = track.get('album').get('name')
        row = [name, track_id, track_name, album_id, album_name]
        table_data.append(row)
    return table_data
        



`get_tracks_table_data` gets the data for `Tracks`

In [9]:
def get_tracks_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Tracks
    """
# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []

    for item in items:
    # Get the track obj
        track = item.get('track')

    # Get track info from track
        track_id = track.get('id')
        track_name = track.get('name')
        album_id = track.get('album').get('id')
        duration_ms = track.get('duration_ms')
        explicit = track.get('explicit')
        current_popularity = track.get('popularity')
        row = (track_id, track_name, album_id, duration_ms, explicit, current_popularity)
        table_data.append(row)
    
# Use track_id to pull audio features
    get_ids = lambda tup: tup[0]
    ids = [get_ids(tup) for tup in table_data]
    audio_features = get_audio_features(sp, ids)

# update each row in table_data
    table_data_length = len(table_data)
    for index in range(table_data_length):
        row = table_data[index]
        track_id = row[0]
        track_audio_features = audio_features.get(track_id)
        acousticness = track_audio_features.get('acousticness')
        danceability = track_audio_features.get('danceability')
        instrumentalness = track_audio_features.get('instrumental')
        key_signature = track_audio_features.get('key')
        mode = track_audio_features.get('mode')
        tempo = track_audio_features.get('tempo')
        time_signature = track_audio_features.get('time_signature')
        valence = track_audio_features.get('valence')
        audio_feature_attributes = (acousticness, danceability, instrumentalness,
                                    key_signature, mode, tempo, time_signature,
                                    valence)
        updated_row = row + audio_feature_attributes
        table_data[index] = updated_row
    return table_data

`get_albums_table_data` gets the data for `Albums`

In [10]:
def get_albums_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Albums
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
# table_data is a dict to ensure return of unique album IDs since multiple tracks from an album
# can be in a playlist
    table_data = {}
    for item in items:
    # Get the track obj
        track = item.get('track')
        album = track.get('album')
        album_id = album.get('id')
        album_name = album.get('name')
        total_tracks = album.get('total_tracks')
        date_released = album.get('release_date')
        row = (album_id, album_name, total_tracks, date_released)
        table_data.update({album_id: row})

    table_data = list(table_data.values())
    return table_data

`get_AlbumArtists_table_data` gets the data for `AlbumArtists`

In [24]:
def get_AlbumArtists_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for AlbumArtists
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        album = track.get('album')
        album_id = album.get('id')
    
    # artists is a list of artist obj
        artists = album.get('artists')
        artists_ids = [artist.get('id') for artist in artists]
        for artist_id in artists_ids:
            row = (album_id, artist_id)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_TrackArtists_table_data` gets the data for `TrackArtists`

In [23]:
def get_TrackArtists_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for TrackArtists
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        track_id = track.get('id')
        
    # artists is a list of artist obj
        artists = track.get('artists')
        artists_ids = [artist.get('id') for artist in artists]
        
        for artist_id in artists_ids:
            row = (track_id, artist_id)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_TrackAvailableMarkets_table_data` gets the data for `TrackAvailableMarkets`

In [29]:
def get_TrackAvailableMarkets_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for TrackAvailableMarkets
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        track_id = track.get('id')
        available_markets = track.get('available_markets')
        
        for market in available_markets:
            row = (track_id, market)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_GenreArtists_table_data` gets the data for `GenreArtists`

In [61]:
def get_GenreArtists_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
        This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for GenreArtists
    """
# List of items
    items = playlist_data.get("items")
   
# Table Data
    table_data = []

# Grab all unique artist ids to get their genre later
    unique_artist_ids = []
    for item in items:
    # Get the track obj
        track = item.get('track')

    # artists is a list of artist obj
        artists = track.get('artists')
    # a dictionary of artist ids and their corresponding genres
        artist_ids = [artist.get('id') for artist in artists]
        unique_artist_ids += artist_ids

    unique_artist_ids = set(unique_artist_ids)
    unique_artist_ids = list(unique_artist_ids)

# artists returns a dictionary with one kvp "artists" containing each artist obj
    artist_genre_data = sp.artists(unique_artist_ids).get('artists')

    for artist_obj in artist_genre_data:
        artist_id = artist_obj.get('id')
        genres = artist_obj.get('genres')
    # create rows for each genre for an artist id
        rows = [(artist_id, genre) for genre in genres]
        table_data += rows
    
    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_Artists_table_data` gets the data for `Artists`

In [65]:
def get_Artists_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
        This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for GenreArtists
    """
# List of items
    items = playlist_data.get("items")
   
# Table Data
    table_data = []

# Grab all unique artist ids to get their genre later
    unique_artist_ids = []
    for item in items:
    # Get the track obj
        track = item.get('track')

    # artists is a list of artist obj
        artists = track.get('artists')
    # a dictionary of artist ids and their corresponding genres
        artist_ids = [artist.get('id') for artist in artists]
        unique_artist_ids += artist_ids

    unique_artist_ids = set(unique_artist_ids)
    unique_artist_ids = list(unique_artist_ids)

# artists returns a dictionary with one kvp "artists" containing each artist obj
    artist_genre_data = sp.artists(unique_artist_ids).get('artists')

    for artist_obj in artist_genre_data:
        artist_id = artist_obj.get('id')
        artist_name = artist_obj.get('name')
        nbr_of_followers = artist_obj.get('followers').get('total')
        popularity = artist_obj.get('popularity')
    # create rows for each genre for an artist id
        row = (artist_id, artist_name, nbr_of_followers, popularity)
        table_data.append(row)
    
    table_data = set(table_data)
    table_data = list(table_data)
    return table_data